<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

import os
from PIL import Image
from datasets import Dataset, DatasetDict
from torchvision import transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# 1. التحميل اليدوي للبيانات
print("🔁 تحميل الصور يدويًا...")

data_dir = "/content/Shark_project_split"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def load_images_from_folder(folder_path):
    data = {"image": [], "label": []}
    class_names = sorted(os.listdir(folder_path))
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        for filename in os.listdir(class_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(class_folder, filename)
                try:
                    img = Image.open(image_path).convert("RGB")
                    data["image"].append(transform(img))
                    data["label"].append(label)
                except Exception as e:
                    print(f"❌ خطأ في الصورة: {image_path} — {e}")
    return Dataset.from_dict(data)

dataset = DatasetDict({
    "train": load_images_from_folder(os.path.join(data_dir, "train")),
    "val": load_images_from_folder(os.path.join(data_dir, "val")),
    "test": load_images_from_folder(os.path.join(data_dir, "test")),
})

print("✅ تم تحميل البيانات.")

# 2. إعداد DataLoader
print("🧱 بناء DataLoaders...")
batch_size = 32

#def collate_fn(batch):
 #   images = torch.stack([item["image"] for item in batch])
#    labels = torch.tensor([item["label"] for item in batch])
 #   return images, labels

# from torchvision import transforms

# تأكد من تحويل الصورة إلى Tensor
to_tensor = transforms.ToTensor()

#def collate_fn(batch):
 #   images = torch.stack([to_tensor(item["image"]) if not isinstance(item["image"], torch.Tensor) else item["image"] for item in batch])
  #  labels = torch.tensor([item["label"] for item in batch])
  #  return images, labels

def collate_fn(batch):
    images = torch.stack([item["image"] for item in batch])  # خلاص الصور جاهزة
    labels = torch.tensor([item["label"] for item in batch])
    return images, labels

train_loader = DataLoader(dataset["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["val"], batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(dataset["test"], batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# 3. إعداد الموديل
print("🧠 تحميل موديل ResNet18...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(dataset["train"]["label"]))

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# 4. التدريب
print("🏋️ بدء التدريب...")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total, correct = 0, 0

    for images, labels in tqdm(train_loader, desc=f"🚂 Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total * 100
    print(f"📊 [Epoch {epoch+1}] Loss: {running_loss:.4f} - Acc: {acc:.2f}%")

# 5. التقييم على validation/test
def evaluate(loader, name=""):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total * 100
    print(f"✅ تقييم {name}: {acc:.2f}%")

print("🔍 تقييم على Validation Set:")
evaluate(val_loader, "Validation")

print("🔍 تقييم على Test Set:")
evaluate(test_loader, "Test")

🔁 تحميل الصور يدويًا...
✅ تم تحميل البيانات.
🧱 بناء DataLoaders...
🧠 تحميل موديل ResNet18...
🏋️ بدء التدريب...


🚂 Epoch 1/10:   0%|          | 0/11 [00:02<?, ?it/s]


TypeError: expected Tensor as element 0 in argument 0, but got list

In [8]:
print(type(dataset['train'][0]['image']))

<class 'list'>


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

# -------------------------------------------------------
print("🧹 التحضير: التجهيزات العامة...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------
print("📁 تحميل الصور يدويًا...")

class SharkDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        class_folders = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_folders)}

        for cls_name in class_folders:
            cls_folder = os.path.join(root_dir, cls_name)
            for filename in os.listdir(cls_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(cls_folder, filename)
                    self.samples.append((path, self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# -------------------------------------------------------
print("🧱 بناء DataLoaders...")

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

batch_size = 32

train_dataset = SharkDataset("/content/Shark_project_resized/train", transform)
val_dataset   = SharkDataset("/content/Shark_project_resized/val", transform)
test_dataset  = SharkDataset("/content/Shark_project_resized/test", transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(train_dataset.class_to_idx)
class_names = list(train_dataset.class_to_idx.keys())
print(f"📊 عدد الأصناف: {num_classes} - {class_names}")

# -------------------------------------------------------
print("🧠 تحميل موديل ResNet18...")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 10

# -------------------------------------------------------
print("🏋️ بدء التدريب...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"🚂 Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100
    print(f"📈 Epoch {epoch+1}: Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")

# -------------------------------------------------------
print("💾 حفظ النموذج...")

torch.save(model.state_dict(), "/content/shark_resnet18_trained.pth")
print("✅ النموذج تم حفظه كـ: shark_resnet18_trained.pth")

# -------------------------------------------------------
print("🔍 التقييم النهائي على بيانات الاختبار...")

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total * 100
print(f"🎯 دقة النموذج على بيانات الاختبار: {test_acc:.2f}%")

In [10]:

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

# -------------------------------------------------------
print("🧹 التحضير: التجهيزات العامة...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------------------------------------
print("📁 تحميل الصور يدويًا...")

class SharkDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        class_folders = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(class_folders)}

        for cls_name in class_folders:
            cls_folder = os.path.join(root_dir, cls_name)
            for filename in os.listdir(cls_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    path = os.path.join(cls_folder, filename)
                    self.samples.append((path, self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# -------------------------------------------------------
print("🧱 بناء DataLoaders...")

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

batch_size = 32

train_dataset = SharkDataset("/content/Shark_project_split/train", transform)
val_dataset   = SharkDataset("/content/Shark_project_split/val", transform)
test_dataset  = SharkDataset("/content/Shark_project_split/test", transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

num_classes = len(train_dataset.class_to_idx)
class_names = list(train_dataset.class_to_idx.keys())
print(f"📊 عدد الأصناف: {num_classes} - {class_names}")

# -------------------------------------------------------
print("🧠 تحميل موديل ResNet18...")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
num_epochs = 10

# -------------------------------------------------------
print("🏋️ بدء التدريب...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in tqdm(train_loader, desc=f"🚂 Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total * 100
    print(f"📈 Epoch {epoch+1}: Loss = {train_loss:.4f}, Accuracy = {train_acc:.2f}%")

# -------------------------------------------------------
print("💾 حفظ النموذج...")

torch.save(model.state_dict(), "/content/shark_resnet18_trained.pth")
print("✅ النموذج تم حفظه كـ: shark_resnet18_trained.pth")

# -------------------------------------------------------
print("🔍 التقييم النهائي على بيانات الاختبار...")

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total * 100
print(f"🎯 دقة النموذج على بيانات الاختبار: {test_acc:.2f}%")

🧹 التحضير: التجهيزات العامة...
📁 تحميل الصور يدويًا...
🧱 بناء DataLoaders...
📊 عدد الأصناف: 8 - ['Blacktip_Shark', 'Bull_Shark', 'Great_White_Shark', 'Hammerhead_Shark', 'Mako_Shark', 'Tiger_Shark', 'Whale_Shark', 'Whitetip_Shark']
🧠 تحميل موديل ResNet18...
🏋️ بدء التدريب...


🚂 Epoch 1/10: 100%|██████████| 11/11 [01:33<00:00,  8.54s/it]


📈 Epoch 1: Loss = 1.0506, Accuracy = 64.72%


🚂 Epoch 2/10: 100%|██████████| 11/11 [01:27<00:00,  7.93s/it]


📈 Epoch 2: Loss = 0.1945, Accuracy = 95.04%


...

📈 Epoch 8: Loss = 0.0069, Accuracy = 99.71%


🚂 Epoch 9/10: 100%|██████████| 11/11 [01:25<00:00,  7.79s/it]


📈 Epoch 9: Loss = 0.0038, Accuracy = 100.00%


🚂 Epoch 10/10: 100%|██████████| 11/11 [01:25<00:00,  7.74s/it]


📈 Epoch 10: Loss = 0.0085, Accuracy = 99.71%
💾 حفظ النموذج...
✅ النموذج تم حفظه كـ: shark_resnet18_trained.pth
🔍 التقييم النهائي على بيانات الاختبار...
🎯 دقة النموذج على بيانات الاختبار: 83.54%


In [2]:
!gdown 165LwqivtdzeXwMaj2VeGzgspqdnOiyrq

Downloading...
From (original): https://drive.google.com/uc?id=165LwqivtdzeXwMaj2VeGzgspqdnOiyrq
From (redirected): https://drive.google.com/uc?id=165LwqivtdzeXwMaj2VeGzgspqdnOiyrq&confirm=t&uuid=c6c13681-d77c-4e0f-b1d4-078443ef86d4
To: /content/Shark_project_split.zip
100% 139M/139M [00:02<00:00, 55.5MB/s]


In [3]:
!unzip -q "/content/Shark_project_split.zip" -d "/content/Shark_project_split"

🔧 المرحلة 1: فك الضغط وتثبيت المكتبات...
